The following tutorial is available on the [Wallaroo Github Repository](https://github.com/WallarooLabs/Wallaroo_Tutorials/blob/wallaroo2025.1_tutorials/wallaroo-run-anywhere/inference-on-any-architecture/x86/edge-llm-summarization).

## Summarization Text Edge Deployment Demonstration

This notebook will walk through building a summarization text pipeline in Wallaroo, deploying it to the local cluster for testing, and then publishing it for edge deployment.

This demonstration will focus on deployment to the edge.  The sample model is available at the following URL.  **This model should be downloaded and placed into the `./models` folder before beginning this demonstration.**

[model-auto-conversion_hugging-face_complex-pipelines_hf-summarisation-bart-large-samsun.zip (1.4 GB)](https://storage.googleapis.com/wallaroo-public-data/llm-models/model-auto-conversion_hugging-face_complex-pipelines_hf-summarisation-bart-large-samsun.zip)


This demonstration performs the following:

1. As a Data Scientist in Wallaroo Ops:
    1. Upload a computer vision model to Wallaroo, deploy it in a Wallaroo pipeline, then perform a sample inference.
    2. Publish the pipeline to an Open Container Initiative (OCI) Registry service.  This is configured in the Wallaroo instance.  See [Edge Deployment Registry Guide](https://docs.wallaroo.ai/wallaroo-platform-operations/wallaroo-platform-operations-configure/wallaroo-edge-deployment/) for details on adding an OCI Registry Service to Wallaroo as the Edge Deployment Registry.  This demonstration uses a GitHub repository - see [Introduction to GitHub Packages](https://docs.github.com/en/packages/learn-github-packages/introduction-to-github-packages) for setting up your own package repository using GitHub, which can then be used with this tutorial.
    3. View the pipeline publish details.
2. As a DevOps Engineer in a remote aka edge device:
    1. Deploy the published pipeline as a Wallaroo Inference Server.  This example will use Docker.
    2. Perform a sample inference through the Wallaroo Inference Server with the same data used in the data scientist example.

## References

* [Wallaroo SDK Essentials Guide: Pipeline Edge Publication](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-pipelines/wallaroo-sdk-essentials-pipeline-publication/)

## Data Scientist Pipeline Publish Steps

### Load Libraries

The first step is to import the libraries used in this notebook.

In [ ]:
import wallaroo
from wallaroo.object import EntityNotFoundError

import pyarrow as pa
import pandas as pd
import time

# used to display dataframe information without truncating
from IPython.display import display
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)

### Connect to the Wallaroo Instance through the User Interface

The next step is to connect to Wallaroo through the Wallaroo client.  The Python library is included in the Wallaroo install and available through the Jupyter Hub interface provided with your Wallaroo environment.

This is accomplished using the `wallaroo.Client()` command, which provides a URL to grant the SDK permission to your specific Wallaroo environment.  When displayed, enter the URL into a browser and confirm permissions.  Store the connection into a variable that can be referenced later.

If logging into the Wallaroo instance through the internal JupyterHub service, use `wl = wallaroo.Client()`.  For more information on Wallaroo Client settings, see the [Client Connection guide](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-client/).

In [ ]:
wl = wallaroo.Client(api_endpoint="https://autoscale-uat-gcp.wallaroo.dev",
                     auth_type="sso")

### Create a New Workspace

We'll use the SDK below to create our workspace , assign as our **current workspace**, then display all of the workspaces we have at the moment.  We'll also set up variables for our models and pipelines down the road, so we have one spot to change names to whatever fits your organization's standards best.

To allow this tutorial to be run by multiple users in the same Wallaroo instance, a random 4 character prefix will be added to the workspace, pipeline, and model.  Feel free to set `suffix=''` if this is not required.

In [14]:
workspace_name = f'edge-hf-summarization'
pipeline_name = 'edge-hf-summarization'
model_name = 'hf-summarization'
model_file_name = './models/model-auto-conversion_hugging-face_complex-pipelines_hf-summarisation-bart-large-samsun.zip'

In [15]:
workspace = wl.get_workspace(name=workspace_name, create_if_not_exist=True)

wl.set_current_workspace(workspace)

ConnectTimeout: timed out

### Configure PyArrow Schema

This is required for non-native runtimes for models deployed to Wallaroo.

You can find more info on the available inputs under [TextSummarizationInputs](https://github.com/WallarooLabs/platform/blob/main/conductor/model-auto-conversion/flavors/hugging-face/src/io/pipeline_inputs/text_summarization_inputs.py#L14) or under the [official source code](https://github.com/huggingface/transformers/blob/v4.28.1/src/transformers/pipelines/text2text_generation.py#L241) from `🤗 Hugging Face`.

In [16]:
input_schema = pa.schema([
    pa.field('inputs', pa.string()),
    pa.field('return_text', pa.bool_()),
    pa.field('return_tensors', pa.bool_()),
    pa.field('clean_up_tokenization_spaces', pa.bool_()),
    # pa.field('generate_kwargs', pa.map_(pa.string(), pa.null())), # dictionaries are not currently supported by the engine
])

output_schema = pa.schema([
    pa.field('summary_text', pa.string()),
])

### Upload the Model

When a model is uploaded to a Wallaroo cluster, it is optimized and packaged to make it ready to run as part of a pipeline. In many times, the Wallaroo Server can natively run a model without any Python overhead. In other cases, such as a Python script, a custom Python environment will be automatically generated. This is comparable to the process of "containerizing" a model by adding a small HTTP server and other wrapping around it.

Our pretrained model is in HuggingFace format, which is specified in the `framework` parameter.  The input and output schemas are included as part of the model upload.  For more information, see [Wallaroo SDK Essentials Guide: Model Uploads and Registrations: Hugging Face](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-model-uploads/wallaroo-sdk-model-upload-hugging-face/).

In [17]:
model = wl.upload_model(model_name, 
                        model_file_name, 
                        framework=wallaroo.framework.Framework.HUGGING_FACE_SUMMARIZATION, 
                        input_schema=input_schema, 
                        output_schema=output_schema,
                        convert_wait=False
                        )
model

Name,hf-summarization
Version,3467272a-55e5-480a-bbd6-9965161f0fbf
File Name,model-auto-conversion_hugging-face_complex-pipelines_hf-summarisation-bart-large-samsun.zip
SHA,ee71d066a83708e7ca4a3c07caf33fdc528bb000039b6ca2ef77fa2428dc6268
Status,pending_load_container
Image Path,None
Architecture,x86
Acceleration,none
Updated At,2025-16-Jul 17:00:15
Workspace id,1671
Workspace name,edge-hf-summarization


In [18]:
while model.status() != "ready" and model.status() != "error":
    print(model.status())
    time.sleep(10)
print(model.status())

ready


### Reserve Pipeline Resources

Before deploying an inference engine we need to tell wallaroo what resources it will need.
To do this we will use the wallaroo DeploymentConfigBuilder() and fill in the options listed below to determine what the properties of our inference engine will be.

We will be testing this deployment for an edge scenario, so the resource specifications are kept small -- what's the minimum needed to meet the expected load on the planned hardware.

- cpus - 4 => allow the engine to use 4 CPU cores when running the neural net
- memory - 8Gi => each inference engine will have 8 GB of memory, which is plenty for processing a single image at a time.

In [19]:
deployment_config = wallaroo.DeploymentConfigBuilder() \
    .cpus(0.25).memory('1Gi') \
    .sidekick_cpus(model, 4) \
    .sidekick_memory(model, "8Gi") \
    .build()

### Simulated Edge Deployment

We will now deploy our pipeline into the current Kubernetes environment using the specified resource constraints. This is a "simulated edge" deploy in that we try to mimic the edge hardware as closely as possible.


In [20]:
pipeline = wl.build_pipeline(pipeline_name)
pipeline.clear()
pipeline.add_model_step(model)

pipeline.deploy(deployment_config=deployment_config, wait_for_status=False)

Deployment initiated for edge-hf-summarization. Please check pipeline status.


name,edge-hf-summarization
created,2025-05-16 16:30:25.959276+00:00
last_updated,2025-07-16 17:04:34.315074+00:00
deployed,True
workspace_id,1671
workspace_name,edge-hf-summarization
arch,x86
accel,none
tags,
versions,"05364bc4-3ed1-4234-9db1-f988822b1ad5, 00a6ba29-09e5-4a47-be7c-fe4b5fd49de3, b04113e5-8d48-4163-a815-5160256885b3, f4ba80ab-778a-4970-bdce-8f1d39310416, fb13126a-02a9-4b4d-8976-6b675c070e8f"
steps,hf-summarization


In [21]:
# check the pipeline status before performing an inference

while pipeline.status()['status'] != 'Running':
   time.sleep(15)
   display(pipeline.status()['status'])

pipeline.status()

'Error'

'Error'

'Error'

'Starting'

'Running'

{'status': 'Running',
 'details': [],
 'engines': [{'ip': '10.4.1.15',
   'name': 'engine-7b7668d5-56jhn',
   'status': 'Running',
   'reason': None,
   'details': [],
   'pipeline_statuses': {'pipelines': [{'id': 'edge-hf-summarization',
      'status': 'Running',
      'version': '05364bc4-3ed1-4234-9db1-f988822b1ad5'}]},
   'model_statuses': {'models': [{'model_version_id': 852,
      'name': 'hf-summarization',
      'sha': 'ee71d066a83708e7ca4a3c07caf33fdc528bb000039b6ca2ef77fa2428dc6268',
      'status': 'Running',
      'version': '3467272a-55e5-480a-bbd6-9965161f0fbf'}]}}],
 'engine_lbs': [{'ip': '10.4.8.10',
   'name': 'engine-lb-648945b8b4-t7x94',
   'status': 'Running',
   'reason': None,
   'details': []}],
 'sidekicks': [{'ip': '10.4.0.3',
   'name': 'engine-sidekick-hf-summarization-852-7fcbb74f46-rg8p7',
   'status': 'Running',
   'reason': None,
   'details': [],
   'statuses': '\n'}]}

### Run Sample Inference

A single inference using sample input data is prepared below.  We'll run through it to verify the pipeline inference is working.

In [22]:
input_data = {
        "inputs": ["LinkedIn (/lɪŋktˈɪn/) is a business and employment-focused social media platform that works through websites and mobile apps. It launched on May 5, 2003. It is now owned by Microsoft. The platform is primarily used for professional networking and career development, and allows jobseekers to post their CVs and employers to post jobs. From 2015 most of the company's revenue came from selling access to information about its members to recruiters and sales professionals. Since December 2016, it has been a wholly owned subsidiary of Microsoft. As of March 2023, LinkedIn has more than 900 million registered members from over 200 countries and territories. LinkedIn allows members (both workers and employers) to create profiles and connect with each other in an online social network which may represent real-world professional relationships. Members can invite anyone (whether an existing member or not) to become a connection. LinkedIn can also be used to organize offline events, join groups, write articles, publish job postings, post photos and videos, and more"], # required
        "return_text": [True], # optional: using the defaults, similar to not passing this parameter
        "return_tensors": [False], # optional: using the defaults, similar to not passing this parameter
        "clean_up_tokenization_spaces": [False], # optional: using the defaults, similar to not passing this parameter
}
dataframe = pd.DataFrame(input_data)
dataframe

,inputs,return_text,return_tensors,clean_up_tokenization_spaces
0,"LinkedIn (/lɪŋktˈɪn/) is a business and employment-focused social media platform that works through websites and mobile apps. It launched on May 5, 2003. It is now owned by Microsoft. The platform is primarily used for professional networking and career development, and allows jobseekers to post their CVs and employers to post jobs. From 2015 most of the company's revenue came from selling access to information about its members to recruiters and sales professionals. Since December 2016, it has been a wholly owned subsidiary of Microsoft. As of March 2023, LinkedIn has more than 900 million registered members from over 200 countries and territories. LinkedIn allows members (both workers and employers) to create profiles and connect with each other in an online social network which may represent real-world professional relationships. Members can invite anyone (whether an existing member or not) to become a connection. LinkedIn can also be used to organize offline events, join groups, write articles, publish job postings, post photos and videos, and more",True,False,False


In [23]:
deploy_url = pipeline._deployment._url()

headers = wl.auth.auth_header()

headers['Content-Type']='application/json; format=pandas-records'
# headers['Content-Type']='application/json; format=pandas-records'
headers['Accept']='application/json; format=pandas-records'

dataFile = './data/test_summarization.df.json'

In [24]:
!curl -X POST {deploy_url} \
     -H "Authorization:{headers['Authorization']}" \
     -H "Content-Type:{headers['Content-Type']}" \
     -H "Accept:{headers['Accept']}" \
     --data-binary @{dataFile}

[{"time":1752685799791,"in":{"clean_up_tokenization_spaces":false,"inputs":"LinkedIn (/lɪŋktˈɪn/) is a business and employment-focused social media platform that works through websites and mobile apps. It launched on May 5, 2003. It is now owned by Microsoft. The platform is primarily used for professional networking and career development, and allows jobseekers to post their CVs and employers to post jobs. From 2015 most of the company's revenue came from selling access to information about its members to recruiters and sales professionals. Since December 2016, it has been a wholly owned subsidiary of Microsoft. As of March 2023, LinkedIn has more than 900 million registered members from over 200 countries and territories. LinkedIn allows members (both workers and employers) to create profiles and connect with each other in an online social network which may represent real-world professional relationships. Members can invite anyone (whether an existing member or not) to become a conne

### Undeploy the Pipeline

Just to clear up resources, we'll undeploy the pipeline.

In [25]:
pipeline.undeploy()

name,edge-hf-summarization
created,2025-05-16 16:30:25.959276+00:00
last_updated,2025-07-16 17:04:34.315074+00:00
deployed,False
workspace_id,1671
workspace_name,edge-hf-summarization
arch,x86
accel,none
tags,
versions,"05364bc4-3ed1-4234-9db1-f988822b1ad5, 00a6ba29-09e5-4a47-be7c-fe4b5fd49de3, b04113e5-8d48-4163-a815-5160256885b3, f4ba80ab-778a-4970-bdce-8f1d39310416, fb13126a-02a9-4b4d-8976-6b675c070e8f"
steps,hf-summarization


### Publish the Pipeline for Edge Deployment

It worked! For a demo, we'll take working once as "tested". So now that we've tested our pipeline, we are ready to publish it for edge deployment.

Publishing it means assembling all of the configuration files and model assets and pushing them to an Open Container Initiative (OCI) repository set in the Wallaroo instance as the Edge Registry service.  DevOps engineers then retrieve that image and deploy it through Docker, Kubernetes, or similar deployments.

See [Edge Deployment Registry Guide](https://docs.wallaroo.ai/wallaroo-platform-operations/wallaroo-platform-operations-configure/wallaroo-edge-deployment/) for details on adding an OCI Registry Service to Wallaroo as the Edge Deployment Registry.

This is done through the SDK command `wallaroo.pipeline.publish(deployment_config)`.

### Publish Example

We will now publish the pipeline to our Edge Deployment Registry with the `pipeline.publish(deployment_config)` command.  `deployment_config` is an optional field that specifies the pipeline deployment.  This can be overridden by the DevOps engineer during deployment.

In [26]:
## This may still show an error status despite but if both containers show running it should be good to go
pipeline.publish(deployment_config)

Waiting for pipeline publish... It may take up to 600 sec.
Pipeline is publishing............................ Published.


ID,110
Pipeline Name,edge-hf-summarization
Pipeline Version,35b4ff87-a0ae-4c63-8766-7c20f9123cd0
Status,Published
Workspace Id,1671
Workspace Name,edge-hf-summarization
Edges,
Engine URL,us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/engines/proxy/wallaroo/ghcr.io/wallaroolabs/fitzroy-mini:v2025.1.0-6245
Pipeline URL,us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/pipelines/edge-hf-summarization:35b4ff87-a0ae-4c63-8766-7c20f9123cd0
Helm Chart URL,oci://us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/charts/edge-hf-summarization
Helm Chart Reference,us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/charts@sha256:955d1b818df76281565899b5fcd19c0ad5a218c49aff915ce570bce9bc164d6e


### List Published Pipeline

The method `wallaroo.client.list_pipelines()` shows a list of all pipelines in the Wallaroo instance, and includes the `published` field that indicates whether the pipeline was published to the registry (`True`), or has not yet been published (`False`).

In [27]:
wl.list_pipelines(workspace_name=workspace_name)

name,created,last_updated,deployed,workspace_id,workspace_name,arch,accel,tags,versions,steps,published
edge-hf-summarization,2025-16-May 16:30:25,2025-16-Jul 17:10:45,False,1671,edge-hf-summarization,x86,none,,"35b4ff87-a0ae-4c63-8766-7c20f9123cd0, 05364bc4-3ed1-4234-9db1-f988822b1ad5, 00a6ba29-09e5-4a47-be7c-fe4b5fd49de3, b04113e5-8d48-4163-a815-5160256885b3, f4ba80ab-778a-4970-bdce-8f1d39310416, fb13126a-02a9-4b4d-8976-6b675c070e8f",hf-summarization,True


### List Publishes from a Pipeline

All publishes created from a pipeline are displayed with the `wallaroo.pipeline.publishes` method.  The `pipeline_version_id` is used to know what version of the pipeline was used in that specific publish.  This allows for pipelines to be updated over time, and newer versions to be sent and tracked to the Edge Deployment Registry service.

#### List Publishes Parameters

N/A

#### List Publishes Returns

A List of the following fields:

| Field | Type | Description |
|---|---|---|
| id | integer | Numerical Wallaroo id of the published pipeline. |
| pipeline_version_id | integer | Numerical Wallaroo id of the pipeline version published. |
| engine_url | string | The URL of the published pipeline engine in the edge registry. |
| pipeline_url | string | The URL of the published pipeline in the edge registry. |
| created_by | string | The email address of the user that published the pipeline.
| Created At | DateTime | When the published pipeline was created. |
| Updated At | DateTime | When the published pipeline was updated. |

In [28]:
pipeline.publishes()

id,Pipeline Name,Pipeline Version,Workspace Id,Workspace Name,Edges,Engine URL,Pipeline URL,Created By,Created At,Updated At
53,edge-hf-summarization,b04113e5-8d48-4163-a815-5160256885b3,1671,edge-hf-summarization,,us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/engines/proxy/wallaroo/ghcr.io/wallaroolabs/fitzroy-mini:v2025.1.0-main-6139,us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/pipelines/edge-hf-summarization:b04113e5-8d48-4163-a815-5160256885b3,john.hummel@wallaroo.ai,2025-16-May 16:36:48,2025-16-May 16:36:48
110,edge-hf-summarization,35b4ff87-a0ae-4c63-8766-7c20f9123cd0,1671,edge-hf-summarization,,us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/engines/proxy/wallaroo/ghcr.io/wallaroolabs/fitzroy-mini:v2025.1.0-6245,us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/pipelines/edge-hf-summarization:35b4ff87-a0ae-4c63-8766-7c20f9123cd0,john.hummel@wallaroo.ai,2025-16-Jul 17:10:47,2025-16-Jul 17:10:47


## DevOps - Pipeline Edge Deployment

Once a pipeline is deployed to the Edge Registry service, it can be deployed in environments such as Docker, Kubernetes, or similar container running services by a DevOps engineer.  For our example, we will use the `docker run` command output during the pipeline publish.

## Edge Deployed Pipeline API Endpoints

Once deployed, we can check the pipelines and models available.  We'll use a `curl` command, but any HTTP based request will work the same way.

The endpoint `/pipelines` returns:

* **id** (*String*):  The name of the pipeline.
* **status** (*String*):  The status as either `Running`, or `Error` if there are any issues.

```bash
curl localhost:8080/pipelines
{"pipelines":[{"id":"edge-cv-retail","status":"Running"}]}
```

In [2]:
!curl testboy.local:8080/pipelines

{"pipelines":[{"id":"edge-hf-summarization","status":"Running"}]}

The endpoint `/models` returns a List of models with the following fields:

* **name** (*String*): The model name.
* **sha** (*String*): The sha hash value of the ML model.
* **status** (*String*):  The status of either Running or Error if there are any issues.
* **version** (*String*):  The model version.  This matches the version designation used by Wallaroo to track model versions in UUID format.

```bash
curl localhost:8080/models
{"models":[{"name":"resnet-50","sha":"c6c8869645962e7711132a7e17aced2ac0f60dcdc2c7faa79b2de73847a87984","status":"Running","version":"693e19b5-0dc7-4afb-9922-e3f7feefe66d"}]}
```

In [3]:
!curl testboy.local:8080/models

{"models":[{"name":"hf-summarization","version":"add06bd2-054e-4562-b25e-ee692c6e472c","sha":"ee71d066a83708e7ca4a3c07caf33fdc528bb000039b6ca2ef77fa2428dc6268","status":"Running"}]}

### Edge Inference Endpoint

The inference endpoint takes the following pattern:

* `/pipelines/infer`

Wallaroo inference endpoint URLs accept the following data inputs through the `Content-Type` header:

* `Content-Type: application/vnd.apache.arrow.file`: For Apache Arrow tables.
* `Content-Type: application/json; format=pandas-records`: For pandas DataFrame in record format.

Once deployed, we can perform an inference through the deployment URL.

The endpoint returns `Content-Type: application/json; format=pandas-records` by default with the following fields:

* **check_failures** (*List[Integer]*): Whether any validation checks were triggered.  For more information, see [Wallaroo SDK Essentials Guide: Pipeline Management: Anomaly Testing](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-pipelines/wallaroo-sdk-essentials-pipeline/#anomaly-testing).
* **elapsed** (*List[Integer]*): A list of time in nanoseconds for:
  * [0] The time to serialize the input.
  * [1...n] How long each step took.
* **model_name** (*String*): The name of the model used.
* **model_version** (*String*): The version of the model in UUID format.
* **original_data**: The original input data.  Returns `null` if the input may be too long for a proper return.
* **outputs** (*List*): The outputs of the inference result separated by data type, where each data type includes:
  * **data**: The returned values.
  * **dim** (*List[Integer]*): The dimension shape returned.
  * **v** (*Integer*): The vector shape of the data.
* **pipeline_name**  (*String*): The name of the pipeline.
* **shadow_data**: Any shadow deployed data inferences in the same format as **outputs**.
* **time** (*Integer*): The time since UNIX epoch.

In [12]:
import json
import requests
import pandas as pd

# set the content type and accept headers
headers = {
    'Content-Type': 'application/json; format=pandas-records'
}

# Submit arrow file
dataFile="./data/test_summarization.df.json"

data = json.load(open(dataFile))

host = 'http://testboy.local:8080'

deployurl = f'{host}/infer'

response = requests.post(
                    deployurl, 
                    headers=headers, 
                    json=data, 
                    verify=True
                )

# display(response)
display(pd.DataFrame(response.json()).loc[0, ['outputs']][0][0]['String']['data'][0])


'LinkedIn is a business and employment-focused social media platform that works through websites and mobile apps. It launched on May 5, 2003. LinkedIn allows members (both workers and employers) to create profiles and connect with each other in an online social network which may represent real-world professional relationships.'